In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cluster import DBSCAN
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iris/Iris.csv
/kaggle/input/iris/database.sqlite
/kaggle/input/headbrain/headbrain.csv
/kaggle/input/ccdata/CC GENERAL.csv


In [2]:
def fetch_data(filename):
    """
    This function is used to upload training file and convert into dataframe.
    :param: filename, :type: .csv
    :return: df
    :return type: class 'pandas.core.frame.DataFrame'
    """
    path = '../input/'+filename+'.csv'
    df = pd.read_csv(path)
    return df

In [3]:
file = "ccdata/CC GENERAL"
file2 = "iris/Iris"
file3 = "headbrain/headbrain"
df = fetch_data(file)

In [4]:
df.shape

(8950, 18)

In [5]:
df.isnull().sum()

CUST_ID                               0
BALANCE                               0
BALANCE_FREQUENCY                     0
PURCHASES                             0
ONEOFF_PURCHASES                      0
INSTALLMENTS_PURCHASES                0
CASH_ADVANCE                          0
PURCHASES_FREQUENCY                   0
ONEOFF_PURCHASES_FREQUENCY            0
PURCHASES_INSTALLMENTS_FREQUENCY      0
CASH_ADVANCE_FREQUENCY                0
CASH_ADVANCE_TRX                      0
PURCHASES_TRX                         0
CREDIT_LIMIT                          1
PAYMENTS                              0
MINIMUM_PAYMENTS                    313
PRC_FULL_PAYMENT                      0
TENURE                                0
dtype: int64

In [6]:
df = df.dropna()
df = df.reset_index(drop=True)

In [7]:
df

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.40,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.00,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.00,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10005,817.714335,1.000000,16.00,16.00,0.00,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12
4,C10006,1809.828751,1.000000,1333.28,0.00,1333.28,0.000000,0.666667,0.000000,0.583333,0.000000,0,8,1800.0,1400.057770,2407.246035,0.000000,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8631,C19184,5.871712,0.500000,20.90,20.90,0.00,0.000000,0.166667,0.166667,0.000000,0.000000,0,1,500.0,58.644883,43.473717,0.000000,6
8632,C19186,28.493517,1.000000,291.12,0.00,291.12,0.000000,1.000000,0.000000,0.833333,0.000000,0,6,1000.0,325.594462,48.886365,0.500000,6
8633,C19188,23.398673,0.833333,144.40,0.00,144.40,0.000000,0.833333,0.000000,0.666667,0.000000,0,5,1000.0,81.270775,82.418369,0.250000,6
8634,C19189,13.457564,0.833333,0.00,0.00,0.00,36.558778,0.000000,0.000000,0.000000,0.166667,2,0,500.0,52.549959,55.755628,0.250000,6


In [8]:
df.dtypes

CUST_ID                              object
BALANCE                             float64
BALANCE_FREQUENCY                   float64
PURCHASES                           float64
ONEOFF_PURCHASES                    float64
INSTALLMENTS_PURCHASES              float64
CASH_ADVANCE                        float64
PURCHASES_FREQUENCY                 float64
ONEOFF_PURCHASES_FREQUENCY          float64
PURCHASES_INSTALLMENTS_FREQUENCY    float64
CASH_ADVANCE_FREQUENCY              float64
CASH_ADVANCE_TRX                      int64
PURCHASES_TRX                         int64
CREDIT_LIMIT                        float64
PAYMENTS                            float64
MINIMUM_PAYMENTS                    float64
PRC_FULL_PAYMENT                    float64
TENURE                                int64
dtype: object

In [9]:
if "Species" in df.columns:
    df = df.drop("Species",axis=1)
    df = df.drop("Id",axis=1)
elif "CUST_ID" in df.columns:
    df = df.drop("CUST_ID",axis=1)
else:
    pass

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)
df=scaler.transform(df)

In [11]:
df

array([[-0.74462486, -0.37004679, -0.42918384, ..., -0.30550763,
        -0.53772694,  0.35518066],
       [ 0.76415211,  0.06767893, -0.47320819, ...,  0.08768873,
         0.21238001,  0.35518066],
       [ 0.42660239,  0.50540465, -0.11641251, ..., -0.09990611,
        -0.53772694,  0.35518066],
       ...,
       [-0.75297728, -0.29709491, -0.40657175, ..., -0.32957217,
         0.30614422, -4.22180042],
       [-0.75772142, -0.29709491, -0.47320819, ..., -0.34081076,
         0.30614422, -4.22180042],
       [-0.58627829, -1.09958965,  0.03129519, ..., -0.32709767,
        -0.53772694, -4.22180042]])

In [12]:
"""
def fit_dbscan(eps,samples,max_amonaly_percentage):
    dbscan = DBSCAN(eps=eps, min_samples=samples)
    result = dbscan.fit(df)
    print(result.labels_)
    labels = result.labels_ 
    print(np.unique(labels))
    labelsdf = pd.DataFrame(labels)
    vals = labelsdf.value_counts()
    print(vals)
    print("iteration complete")#First clusters are created with user defined eps,minsamples
    for i in vals.index:
        print("cluster",int(i[0])," counts",vals[i])
        count = vals[i]
        cluster = int(i[0])
        if cluster == -1:
            if count>((labels.size/100)*max_amonaly_percentage):
                #If anomaly size is greater than max allowed, then increase eps to reduce anomaly
                eps = eps+0.2
                #If some clusters strictly follwing minsamples as 2, then remove them by increasing samples 
                if vals.min() <= 2:
                    samples = samples+1
            
                labels, eps, samples, vals = fit_dbscan(eps,samples,max_amonaly_percentage)
                break
    return labels, eps, samples, vals
"""

'\ndef fit_dbscan(eps,samples,max_amonaly_percentage):\n    dbscan = DBSCAN(eps=eps, min_samples=samples)\n    result = dbscan.fit(df)\n    print(result.labels_)\n    labels = result.labels_ \n    print(np.unique(labels))\n    labelsdf = pd.DataFrame(labels)\n    vals = labelsdf.value_counts()\n    print(vals)\n    print("iteration complete")#First clusters are created with user defined eps,minsamples\n    for i in vals.index:\n        print("cluster",int(i[0])," counts",vals[i])\n        count = vals[i]\n        cluster = int(i[0])\n        if cluster == -1:\n            if count>((labels.size/100)*max_amonaly_percentage):\n                #If anomaly size is greater than max allowed, then increase eps to reduce anomaly\n                eps = eps+0.2\n                #If some clusters strictly follwing minsamples as 2, then remove them by increasing samples \n                if vals.min() <= 2:\n                    samples = samples+1\n            \n                labels, eps, sample

In [13]:
#eps = 1.0
#samples = 2
#max_amonaly_percentage = 5 #How much percent of anomaly should be allowed in clusters, reduces values in cluster -1
#labels, eps, samples, vals = fit_dbscan(eps,samples,max_amonaly_percentage)

In [14]:
#labelsdf2 = pd.DataFrame(labels)
#print("Final EPS: ",eps," Final Min-samples: ",samples)
#labelsdf2.value_counts()

In [15]:
def fit_dbscan2(eps,samples,anomaly_count2):
    dbscan = DBSCAN(eps=eps, min_samples=samples)
    result = dbscan.fit(df)
    labels = result.labels_ 
    print(np.unique(labels))
    labelsdf = pd.DataFrame(labels)
    vals = labelsdf.value_counts()
    print(vals)
    print("iteration complete")#First clusters are created with user defined eps,minsamples
    anomaly_count = vals[-1]
    print("anomaly_count ",anomaly_count)
    print("anomaly_count2 ",anomaly_count2)
    if anomaly_count2 == 0:
        eps=eps+0.05
        labels, eps, samples, vals = fit_dbscan2(eps,samples,anomaly_count)
    elif (anomaly_count2-anomaly_count)>1: #Compare anomaly counts of n and n-1 iteration
        anomaly_count2=anomaly_count
        eps=eps+0.05
        labels, eps, samples, vals = fit_dbscan2(eps,samples,anomaly_count2)
    else:
        return labels, eps, samples, vals
    return labels, eps, samples, vals

In [16]:
eps = 1.0
samples = 2
anomaly_count2=0
labels, eps, samples, vals = fit_dbscan2(eps,samples,anomaly_count2)

[ -1   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16
  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88
  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106
 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160
 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178
 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203]
 0      5701
-1      2376
 5        22
 43       19
 25       11
        ... 
 83        2
 85        2
 86        2
 88        2
 203       2
Length: 205, dtype: int6

In [17]:
labelsdf2 = pd.DataFrame(labels)
print("Final EPS: ",eps," Final Min-samples: ",samples)
labelsdf2.value_counts()

Final EPS:  3.649999999999995  Final Min-samples:  2


 0     8523
-1       92
 7        3
 1        2
 2        2
 3        2
 4        2
 5        2
 6        2
 8        2
 9        2
 10       2
dtype: int64